In [ ]:
%pip install -r requirements.txt

In [ ]:
import time
import requests
from datetime import datetime, timedelta
from pathlib import Path
import requests
import json

In [ ]:
# Load app IDs from file
app_ids = []
with open("app_ids.txt", "r") as f:
    app_ids = [line.strip() for line in f]

# API parameters
base_url = "https://store.steampowered.com/appreviews/"
time_interval = timedelta(hours=24)
end_time = datetime.now()
start_time = datetime(2024, 1, 1, 0, 0, 0)  # Adjust if needed

print(f"Start time: {start_time}")
print(f"End time: {end_time}")
print(start_time.timestamp(), end_time.timestamp())

output_path = Path("app_ids_with_reviews.txt")
filtered_app_ids = []
selected_reviews = []

session = requests.Session()
session.mount("https://", requests.adapters.HTTPAdapter(max_retries=3))

def get_user_reviews(app_id, params, max_retries=3):
    """Fetches reviews from Steam API with retry on timeout."""
    for attempt in range(max_retries):
        try:
            response = session.get(f"{base_url}{app_id}", params=params, timeout=10, verify=True)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.Timeout:
            print(f"Timeout for app {app_id}. Retrying in 10 seconds... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(10)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for app {app_id}: {e}")
            return None  # Stop retrying on non-timeout errors
    return None  # If all retries fail

def has_min_reviews(app_id, total_reviews, min_reviews=4):
    """Checks if an app has at least `min_reviews`."""
    return total_reviews >= min_reviews

# Process each app ID
for app_id in app_ids:
    print(f"\nGetting reviews for app_id: {app_id}")
    params = {
        "json": 1,
        "language": "turkish",
        "cursor": "*",
        "num_per_page": 100,
        "filter": "recent",
    }

    page = 0
    passed_start_time = False
    passed_end_time = False

    while page < 10 and (not passed_start_time or not passed_end_time):
        reviews_response = get_user_reviews(app_id, params)

        if not reviews_response or reviews_response.get("success") != 1:
            print("Request failed or no success response.")
            break

        total_reviews = reviews_response.get("query_summary", {}).get("num_reviews", 0)
        # print(f"Total reviews: {total_reviews}")
        print(f"Total reviews: {total_reviews}")
        if total_reviews == 0:
            print("No reviews found for this app.")
            break

        if has_min_reviews(app_id, total_reviews, min_reviews=4):
            if app_id not in filtered_app_ids:
                filtered_app_ids.append(app_id)

            for review in reviews_response["reviews"]:
                timestamp_created = review["timestamp_created"]
                if timestamp_created > end_time.timestamp():
                    continue
                if timestamp_created < start_time.timestamp():
                    passed_start_time = True
                    break

                recommendation_id = review['recommendationid']
                timestamp_created = review['timestamp_created']
                timestamp_updated = review['timestamp_updated']
                # extract the useful (to me) data
                # Add APP ID to the review 
                review['app_id'] = app_id
                author_steamid = review['author']['steamid']        # will automatically redirect to the profileURL if any
                review_text = review['review']
                voted_up = review['voted_up']
                votes_up = review['votes_up']
                votes_funny = review['votes_funny']
                weighted_vote_score = review['weighted_vote_score']
                steam_purchase = review['steam_purchase']
                received_for_free = review['received_for_free']
                written_during_early_access = review['written_during_early_access']
                

                my_review_dict = {
                    'app_id': app_id,
                    'recommendationid': recommendation_id,
                    'author_steamid': author_steamid,

                    'review_text': review_text,
                    'timestamp_created': timestamp_created,
                    'timestamp_updated': timestamp_updated,

                    'voted_up': voted_up,
                    'votes_up': votes_up,
                    'votes_funny': votes_funny,
                    'weighted_vote_score': weighted_vote_score,
                    'steam_purchase': steam_purchase,
                    'received_for_free': received_for_free,
                    'written_during_early_access': written_during_early_access,
                }
                review["app_id"] = app_id
                selected_reviews.append(review)

        # Get next page cursor
        cursor = reviews_response.get("cursor", "")
        if not cursor:
            print("Reached the last page.")
            break
        
        params["cursor"] = cursor
        print(f"Moving to page {page + 1}. Cursor: {cursor}")
        page += 1
        time.sleep(2)

    time.sleep(2)  # Wait before fetching the next app

# Save filtered app IDs
with open(output_path, "w") as f:
    for app_id in filtered_app_ids:
        # Append app ID to the file
        f.write(f"{app_id}\n")

print("Finished processing all app reviews.")